In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
import re
from datetime import datetime

In [55]:
area = "bangkok"
data = pd.read_csv("data/airbnb_" + area + "_cleaned.csv", index_col=0)

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [56]:
data["property_type"].value_counts().head(30)

Entire rental unit                     4558
Entire condominium (condo)             3492
Private room in rental unit            1830
Private room in condominium (condo)     899
Private room in residential home        646
Room in boutique hotel                  638
Room in hotel                           528
Private room in townhouse               524
Private room in hostel                  500
Entire serviced apartment               456
Entire residential home                 447
Shared room in hostel                   397
Private room in bed and breakfast       311
Entire townhouse                        310
Room in hostel                          296
Private room in serviced apartment      259
Room in aparthotel                      175
Room in serviced apartment              153
Private room                            125
Private room in guesthouse              104
Room in bed and breakfast                95
Entire loft                              90
Private room in guest suite     

In [5]:
# keep if property type is Apartment, House or Townhouse
data = data.loc[lambda x: x["property_type"].isin(["Apartment", "House", "Townhouse"])]

In [6]:
# rename Townhouse to House

data["property_type"] = np.where(
    data["property_type"] == "Townhouse", "House", data["property_type"]
)
data["f_property_type"] = data["property_type"].astype("category")

In [57]:
data["room_type"].value_counts()

Entire home/apt    9660
Private room       6291
Hotel room          960
Shared room         670
Name: room_type, dtype: int64

In [8]:
#Room type as factor

data["f_room_type"] = data["room_type"].astype("category")

In [9]:
# Rename roomt type because it is too long
data["f_room_type2"] = data["f_room_type"].map(
    {
        "Entire home/apt": "Entire/Apt",
        "Private room": "Private",
        "Shared room": "Shared",
    }
)

In [61]:
data.columns[:50]

Index(['scrape_id', 'host_id', 'host_name', 'host_since', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'la

In [66]:
data.columns[50:100]

Index(['review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'Unnamed: 61', '\ hdtv', '\ hdtv with apple tv',
       '\ hdtv with chromecast', '\ hdtv with netflix',
       '\ hdtv with standard cable', '\ tv', '\ tv with chromecast',
       '\ tv with netflix', '\ tv with standard cable', 'air conditioning',
       'airport shuttle', 'aloe vera body soap', 'amazon prime video',
       'and closet', 'and desk', 'and dresser', 'and office chair',
       'and table', 'and wardrobe', 'apple tv', 'aro body soap',
       'aro conditioner', 'aro shampoo', 'baby bath', 'baby monitor',
       'baby safety gates', 'babysitter recommendations', 'backyard',
       'baking sheet', 'barbecue utensils', 'bath

In [65]:
data["beds"]

id
27934       1.0
27979       2.0
28354       1.0
28745       1.0
35780       1.0
           ... 
51828176    1.0
51851215    1.0
51851676    3.0
51854076    3.0
51854776    3.0
Name: beds, Length: 17581, dtype: float64

In [58]:
# cancellation policy as factor
data["cancellation_policy"].value_counts()

KeyError: 'cancellation_policy'

In [11]:
# if cancellation policy is super strict 30 or 60, rename it as strict
data["cancellation_policy"] = np.where(
    (data["cancellation_policy"] == "super_strict_30")
    | (data["cancellation_policy"] == "super_strict_60"),
    "strict",
    data["cancellation_policy"],
)
data["f_cancellation_policy"] = data["cancellation_policy"].astype("category")

In [12]:
# bed_type and neighbourhood_cleansed as factors

data["bed_type"] = np.where(
    data["bed_type"].isin(["Futon", "Pull-out Sofa", "Airbed"]),
    "Couch",
    data["bed_type"],
)

data["f_bed_type"] = data["bed_type"].astype("category")
data["f_neighbourhood_cleansed"] = data["neighbourhood_cleansed"].astype("category")

---------

### Create Numerical variables

In [67]:
data["usd_price_day"] = data["price"]
data["p_host_response_rate"] = data["host_response_rate"].fillna(0).astype(int)
# rename cleaning_fee column

data = data.rename(columns={"cleaning_fee":"usd_cleaning_fee"})

In [70]:
# add new numeric columns from certain columns

numericals = [
    "accommodates",
    "bathrooms",
    "review_scores_rating",
    "number_of_reviews",
    #"guests_included",
    "reviews_per_month",
    #"extra_people",
    "minimum_nights",
    "beds",
]

for col in numericals:
    data["n_" + col] = pd.to_numeric(data[col], errors="coerce")

In [71]:
# create days since first review

data["n_days_since"] = (
    data.calendar_last_scraped.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    - data.first_review.fillna("1950-01-01").apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
).dt.days

data["n_days_since"] = np.where(data.first_review.isnull(), np.nan, data.n_days_since)

In [72]:
# create dummy vars
dummies = data.columns[71:121]

for col in dummies:
    data["d_" + (re.sub("/|\s|-", "", col)).replace("(s)", "s").lower()] = data[col]

In [75]:
# keep columns if contain d_, n_,f_, p_, usd_ and some others

data = pd.concat(
    [
        data.filter(
            regex="^d_.*|^n_.*|^f_.*|^p_.*|^usd_.*",
        ),
        data[
            [
                "price",
                #"id",
                "neighbourhood_cleansed",
                #"cancellation_policy",
                "room_type",
                "property_type",
            ]
        ],
    ],
    axis=1,
)

In [81]:
 data["price"].astype(float)

id
27934         NaN
27979         NaN
28354         NaN
28745       800.0
35780       800.0
            ...  
51828176    350.0
51851215    960.0
51851676      NaN
51854076      NaN
51854776      NaN
Name: price, Length: 17581, dtype: float64

In [76]:
#####################
### look at price ###
#####################

data["price"] = data["price"].str.replace(",", "").astype(float)

data = data.loc[lambda x: x.price < 1000]

AttributeError: Can only use .str accessor with string values!

In [19]:
# Squares and further values to create
data = data.assign(
    n_accommodates2=lambda x: x["n_accommodates"] ** 2,
    ln_accommodates=lambda x: np.log(x["n_accommodates"]),
    ln_accommodates2=lambda x: np.log(x["n_accommodates"]) ** 2,
    ln_beds=lambda x: np.log(x["n_beds"]),
    ln_number_of_reviews=lambda x: np.log(x["n_number_of_reviews"] + 1),
)

/Users/vigadam/.local/share/virtualenvs/da_case_studies-A7kE1pOP/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log


In [20]:
# Pool accomodations with 0,1,2,10 bathrooms

bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 10)], closed="left")
f_bath = pd.cut(data["n_bathrooms"].to_list(), bins, labels=["0", "1", "2"])
f_bath.categories = [0, 1, 2]
data["f_bathroom"] = f_bath

f_bath.value_counts(dropna=False)

0        364
1      41417
2       9628
NaN      237
dtype: int64

In [21]:
# Pool num of reviews to 3 categories: none, 1-51 and >51

bins = pd.IntervalIndex.from_tuples(
    [(0, 1), (1, 51), (51, max(data.n_number_of_reviews))], closed="left"
)
fnor = pd.cut(data["n_number_of_reviews"].to_list(), bins, labels=["0", "1", "2"])
fnor.categories = [0, 1, 2]
data["f_number_of_reviews"] = fnor
data["f_number_of_reviews"].value_counts(dropna=False)

1      32683
0      15741
2       3221
NaN        1
Name: f_number_of_reviews, dtype: int64

In [22]:
# Pool and categorize the number of minimum nights: 1,2,3, 3+

bins = pd.IntervalIndex.from_tuples(
    [(1, 2), (2, 3), (3, max(data.n_minimum_nights))], closed="left"
)
f_min_n = pd.cut(data["n_minimum_nights"].to_list(), bins, labels=["1", "2", "3"])
f_min_n.categories = [1, 2, 3]
data["f_minimum_nights"] = f_min_n
data["f_minimum_nights"].value_counts(dropna=False)

1      19454
3      18075
2      14116
NaN        1
Name: f_minimum_nights, dtype: int64

In [23]:
# Change Infinite values with NaNs
data = data.replace([np.inf, -np.inf], np.nan)

In [24]:
# ------------------------------------------------------------------------------------------------
# where do we have missing variables now?
to_filter = data.isna().sum()
to_filter[to_filter > 0]

usd_cleaning_fee          20017
n_bathrooms                 237
n_review_scores_rating    16501
n_reviews_per_month       15741
n_beds                      167
n_days_since              15741
ln_beds                     168
f_bathroom                  237
f_number_of_reviews           1
f_minimum_nights              1
dtype: int64

In [25]:
# what to do with missing values?
# 1. drop if no target
data = data.loc[lambda x: x.price.notnull()]

In [26]:
# 2. imput when few, not that important
data = data.assign(
    n_bathrooms=lambda x: x["n_bathrooms"].fillna(np.median(x["n_bathrooms"].dropna())),
    n_beds=lambda x: np.where(x["n_beds"].isnull(), x["n_accommodates"], x["n_beds"]),
    f_bathroom=lambda x: x["f_bathroom"].fillna(1),
    f_minimum_nights=lambda x: x["f_minimum_nights"].fillna(1),
    f_number_of_reviews=lambda x: x["f_number_of_reviews"].fillna(1),
    ln_beds=lambda x: x["ln_beds"].fillna(0),
)

In [27]:
# 3. drop columns when many missing not important
data = data.drop(["usd_cleaning_fee", "p_host_response_rate"], axis=1)

In [28]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

n_review_scores_rating    16501
n_reviews_per_month       15741
n_days_since              15741
dtype: int64

In [29]:
# 4. Replace missing variables re reviews with zero, when no review + add flags
data = data.assign(
    flag_days_since=np.multiply(data.n_days_since.isna(), 1),
    n_days_since=data.n_days_since.fillna(np.median(data.n_days_since.dropna())),
    flag_review_scores_rating=np.multiply(data.n_review_scores_rating.isna(), 1),
    n_review_scores_rating=data.n_review_scores_rating.fillna(
        np.median(data.n_review_scores_rating.dropna())
    ),
    flag_reviews_per_month=np.multiply(data.n_reviews_per_month.isna(), 1),
    n_reviews_per_month=data.n_reviews_per_month.fillna(
        np.median(data.n_reviews_per_month.dropna())
    ),
    flag_n_number_of_reviews=np.multiply(data.n_number_of_reviews.isna(), 1),
)

In [30]:
data.flag_days_since.value_counts()

0    35905
1    15741
Name: flag_days_since, dtype: int64

In [31]:
# redo features
# Create variables, measuring the time since: squared, cubic, logs
data = data.assign(
    ln_days_since=lambda x: np.log(x["n_days_since"] + 1),
    ln_days_since2=lambda x: np.log(x["n_days_since"] + 1) ** 2,
    ln_days_since3=lambda x: np.log(x["n_days_since"] + 1) ** 3,
    n_days_since2=lambda x: x["n_days_since"] ** 2,
    n_days_since3=lambda x: x["n_days_since"] ** 3,
    ln_review_scores_rating=lambda x: np.log(x["n_review_scores_rating"]),
)

In [32]:
data.ln_days_since = data["ln_days_since"].fillna(0)
data.ln_days_since2 = data["ln_days_since2"].fillna(0)
data.ln_days_since3 = data["ln_days_since3"].fillna(0)

In [33]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

Series([], dtype: int64)

In [34]:
data.describe()

,n_accommodates,n_bathrooms,n_review_scores_rating,n_number_of_reviews,n_guests_included,n_reviews_per_month,n_extra_people,n_minimum_nights,n_beds,n_days_since,...,flag_days_since,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating
count,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,51646.000000,...,51646.000000,51646.000000,51646.000000,51646.0,51646.000000,51646.000000,51646.000000,5.164600e+04,5.164600e+04,51646.000000
mean,3.057178,1.260708,92.439627,12.350327,1.415773,1.138970,6.665550,3.310266,1.708884,418.132595,...,0.304786,0.319502,0.304786,0.0,5.714039,33.464400,199.676817,2.936171e+05,3.051141e+08,4.520661
std,1.888509,0.527094,8.438353,25.860475,1.044884,1.236552,12.691355,29.083719,1.168387,344.651296,...,0.460321,0.466288,0.460321,0.0,0.902317,9.507929,80.289121,5.640849e+05,9.621195e+08,0.120946
min,1.000000,0.000000,20.000000,0.000000,1.000000,0.010000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2.995732
25%,2.000000,1.000000,92.000000,0.000000,1.000000,0.470000,0.000000,1.000000,1.000000,228.000000,...,0.000000,0.000000,0.000000,0.0,5.433722,29.525335,160.432461,5.198400e+04,1.185235e+07,4.521789
50%,2.000000,1.000000,94.000000,3.000000,1.000000,0.770000,0.000000,2.000000,1.000000,327.000000,...,0.000000,0.000000,0.000000,0.0,5.793014,33.559007,194.407782,1.069290e+05,3.496578e+07,4.543295
75%,4.000000,1.500000,97.000000,12.000000,1.000000,1.170000,10.000000,3.000000,2.000000,504.000000,...,1.000000,1.000000,1.000000,0.0,6.224558,38.745128,241.171311,2.540160e+05,1.280241e+08,4.574711
max,16.000000,8.000000,100.000000,396.000000,16.000000,15.000000,240.000000,5000.000000,16.000000,2722.000000,...,1.000000,1.000000,1.000000,0.0,7.909489,62.560024,494.817853,7.409284e+06,2.016807e+10,4.605170


In [35]:
data.to_csv(data_out + "airbnb_london_workfile_adj.csv", index=False)